In [1]:
class CustomBERTModel(nn.Module):
    def __init__(self, num_labels, vocab):
        self.model = AutoAdapterModel.from_pretrained(BERT_MODEL, config=config)   # self.create_featureful_bert()

        lang_adapter_config = AdapterConfig.load("pfeiffer", reduction_factor=2)
        model.load_adapter("en/wiki@ukp", config=lang_adapter_config)
        model.load_adapter("de/wiki@ukp", config=lang_adapter_config)
        model.add_adapter("taskname")

        # self.encoder = CustomPooler2()
        # self.module1 = MyModule(self.feature_list, vocab)
        self.relation_decoder = nn.Linear(self._decoder_input_size, self.num_classes) #sanity check. same as their function

In [2]:
class CustomBERTModel(nn.Module):
    def __init__(self, num_labels, vocab):
          super(CustomBERTModel, self).__init__()
          self.embedder = self.create_featureful_bert()
        #   self.encoder = CustomPooler2()
        #   self.module1 = MyModule(self.feature_list, vocab)
        #   self.relation_decoder = nn.Linear(self._decoder_input_size, self.num_classes)

In [9]:
from transformers import AutoConfig, AutoAdapterModel

config = AutoConfig.from_pretrained(BERT_MODEL)

model = AutoAdapterModel.from_pretrained(BERT_MODEL, config=config)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertAdapterModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertAdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertAdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


This is the base AutoModel. If you print model before adding the adapters, you can see that the embeddings+12 transformer layers will be shared by En and Zh adapters.

In [10]:
from transformers import AdapterConfig

# Load the language adapters
lang_adapter_config = AdapterConfig.load("pfeiffer", reduction_factor=2)
model.load_adapter("en/wiki@ukp", config=lang_adapter_config)
model.load_adapter("de/wiki@ukp", config=lang_adapter_config)

# Add a new task adapter
model.add_adapter("taskname")

# Add a classification head for our target task
model.add_classification_head("taskname", num_labels=num_labels)

ValueError: Connection error, and we cannot find the requested files in the cached path. Please try again or make sure your Internet connection is on.

Using `train_adapter()`, we tell our model to only train the task adapter in the following. This call will freeze the weights of the pre-trained model and the weights of the language adapters to prevent them from further finetuning.

In [ ]:
model.train_adapter(["disrpt"])

We want the task adapter to be stacked on top of the language adapter, so we have to tell our model to use this setup via the `active_adapters` property.

A stack of adapters is represented by the `Stack` class, which takes the names of the adapters to be stacked as arguments.
Of course, there are various other possibilities to compose adapters beyonde stacking. Learn more about those [in our documentation](https://docs.adapterhub.ml/adapter_composition.html).

In [ ]:
# Unfreeze and activate stack setup
from transformers.adapters.composition import Stack

lang = 'en'
model.active_adapters = Stack(lang, "disrpt")

Great! Now, the input will be passed through the English language adapter first and the COPA task adapter second in every forward pass.

For training, we make use of the `Trainer` class built-in into `transformers`. We configure the training process using a `TrainingArguments` object.

As the dataset splits of English COPA in the SuperGLUE are slightly different, we train on both the train and validation split of the dataset. Later, we will evaluate on the test split of XCOPA.

In [ ]:
# TODO: SHUFFLE print smples: https://discuss.huggingface.co/t/how-to-print-a-few-examples-at-the-beginning-of-training-when-using-trainer/7597/2
# https://github.com/huggingface/datasets/blob/cd3169f3f35afcf73a36a8276113e1881d92e5e0/src/datasets/iterable_dataset.py#L223

In [ ]:
from copy import deepcopy
from transformers import TrainerCallback

class CustomCallback(TrainerCallback):
    
    def __init__(self, trainer) -> None:
        super().__init__()
        self._trainer = trainer
    
    def on_epoch_end(self, args, state, control, **kwargs):
        if control.should_evaluate:
            control_copy = deepcopy(control)
            self._trainer.evaluate(eval_dataset=self._trainer.train_dataset, metric_key_prefix="train@"+lang)
            return control_copy

2023-01-10 14:51:33.856254: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-10 14:51:34.080263: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda-10.1/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda-10.2/targets/x86_64-linux/lib/:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-10 14:51:34.080303: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-10

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, log_loss
from torch.nn import CrossEntropyLoss
import numpy as np

def compute_metrics(pred):
    global num_labels
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    loss_fct = CrossEntropyLoss()
    logits = torch.tensor(pred.predictions)
    labels = torch.tensor(labels)
    loss = loss_fct(logits.view(-1, num_labels), labels.view(-1))
    return {
        'accuracy@'+lang: acc,
        'f1@'+lang: f1,
        'precision@'+lang: precision,
        'recall@'+lang: recall,
        'loss@'+lang: loss,
    }

In [ ]:
from transformers import TrainingArguments, AdapterTrainer
# from datasets import concatenate_datasets
from itertools import chain


def concatenate_datasets(dataset1, dataset2, shuffle=True):
    y_iter = chain(dataset1, dataset2)
    return y_iter
    # tmp = list(yielding(y_iter))
    # random.shuffle(tmp)
    # for i in tmp:
    #     print i

training_args = TrainingArguments(
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=1e-4,
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    output_dir=MODEL_DIR+'_EN',
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    save_total_limit=1,
)

# train_dataset.combine_with_another_dataset(valid_dataset)
# train_dataset = concatenate_datasets([dataset_en["train"], dataset_en["validation"]])

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=en_train_dataset,
    eval_dataset=en_valid_dataset,
    compute_metrics=compute_metrics
)

trainer.add_callback(CustomCallback(trainer)) 

# Train

Start the training 🚀 (this will take a while)

In [ ]:
raise ValueError()

ValueError: 

In [ ]:
train_result = trainer.train()

metrics = train_result.metrics
trainer.log_metrics("train_log", metrics)
trainer.save_metrics("train_log", metrics)

/home/VD/kaveri/anaconda3/envs/py310/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 100
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 14
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Accuracy@en,F1@en,Precision@en,Recall@en,Loss@en
1,No log,3.013073,0.380000,0.060696,0.049729,0.088542,3.013073
1,3.164400,3.061029,0.373843,0.048936,0.041058,0.068333,3.061030
2,3.164400,2.924473,0.330000,0.044665,0.051197,0.071875,2.924473
2,2.956900,2.978199,0.399136,0.043918,0.040088,0.064190,2.978199


***** Running Evaluation *****
  Num examples = 100
  Batch size = 16
/home/VD/kaveri/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 1621
  Batch size = 16
/home/VD/kaveri/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 100
  Batch size = 16
/home/VD/kaveri/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: Undefi

***** train_log metrics *****
  epoch                    =        2.0
  total_flos               =    58323GF
  train_loss               =     3.0606
  train_runtime            = 0:00:50.40
  train_samples_per_second =      3.968
  train_steps_per_second   =      0.278


In [ ]:
# eval_trainer = AdapterTrainer(
#     model=model,
#     args=TrainingArguments(output_dir="./eval_output_en", remove_unused_columns=False,),
#     eval_dataset=en_test_dataset,
#     compute_metrics=compute_metrics,
# )
trainer.evaluate(metric_key_prefix='test_en',
                eval_dataset=en_test_dataset)

***** Running Evaluation *****
  Num examples = 2155
  Batch size = 16


{'test_en_loss': 1.20975661277771,
 'test_en_accuracy@en': 0.6580046403712297,
 'test_en_f1@en': 0.4881558886369518,
 'test_en_precision@en': 0.5719035643638274,
 'test_en_recall@en': 0.45642819587598726,
 'test_en_loss@en': 1.20975661277771,
 'test_en_runtime': 45.5675,
 'test_en_samples_per_second': 47.292,
 'test_en_steps_per_second': 2.963,
 'epoch': 8.0}

## Cross-lingual transfer

With the model and all adapters trained and ready, we can come to the cross-lingual transfer step here. We will evaluate our setup on the Chinese split of the XCOPA dataset.
Therefore, we'll first download the data and preprocess it using the same method as the English dataset:

Next, let's adapt our setup to the new language. We simply replace the English language adapter with the Chinese language adapter we already loaded previously. The task adapter we just trained is kept. Again, we set this architecture using `active_adapters`:

In [ ]:
lang = 'de'
model.active_adapters = Stack(lang, "disrpt")

Finally, let's see how well our adapter setup performs on the new language. We measure the zero-shot accuracy on the test split of the target language dataset. Evaluation is also performed using the built-in `Trainer` class.

### Set training args for de

In [ ]:
training_args = TrainingArguments(
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=1e-4,
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    output_dir=MODEL_DIR+'_DE',
    overwrite_output_dir=False,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    save_total_limit=1,
    # resume_from_checkpoint=MODEL_DIR+'/last-checkpoint',
)

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=de_train_dataset,
    eval_dataset=de_valid_dataset,
    compute_metrics=compute_metrics
)

trainer.add_callback(CustomCallback(trainer)) 

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()

metrics = train_result.metrics
trainer.log_metrics("train_logger", metrics)
trainer.save_metrics("train_logger", metrics)

/home/VD/kaveri/anaconda3/envs/py310/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2164
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2168
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Accuracy@de,F1@de,Precision@de,Recall@de,Loss@de
1,No log,2.225829,0.350277,0.228179,0.275031,0.253067,2.225829
1,2.741200,2.445551,0.331950,0.205794,0.249839,0.246427,2.445552
2,2.741200,1.953777,0.425601,0.309457,0.354098,0.323060,1.953777
2,2.265700,2.291186,0.360996,0.256613,0.261596,0.274415,2.291186
3,2.265700,1.808799,0.455176,0.324228,0.345672,0.354156,1.808799
3,2.074600,2.268173,0.344398,0.236176,0.235492,0.267678,2.268173
4,2.074600,1.698839,0.471349,0.339613,0.397413,0.355348,1.698839
4,1.951100,2.294407,0.327801,0.237641,0.237618,0.260488,2.294407
5,1.951100,1.599388,0.507394,0.385863,0.398206,0.406357,1.599388
5,1.854400,2.266076,0.373444,0.258835,0.261072,0.288853,2.266076


***** Running Evaluation *****
  Num examples = 2164
  Batch size = 8
/home/VD/kaveri/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 241
  Batch size = 8
/home/VD/kaveri/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to AdapterAlpha_lr=AdafactorDefault_plm=bert-base-multilingual-cased_Info=v3_lrtest1_DE/checkpoint-500
Configuration saved in AdapterAlpha_lr=AdafactorDefault_pl

***** train_logger metrics *****
  epoch                    =        8.0
  total_flos               = 37331542GF
  train_loss               =     1.0419
  train_runtime            = 2:21:33.28
  train_samples_per_second =     15.073
  train_steps_per_second   =      0.943


In [ ]:
trainer.evaluate(metric_key_prefix='test_de',
                eval_dataset=de_test_dataset)

***** Running Evaluation *****
  Num examples = 260
  Batch size = 8


/home/VD/kaveri/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/VD/kaveri/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'test_de_loss': 2.554270029067993,
 'test_de_accuracy@de': 0.25384615384615383,
 'test_de_f1@de': 0.1910287697443995,
 'test_de_precision@de': 0.1876885104298849,
 'test_de_recall@de': 0.22257923190174714,
 'test_de_loss@de': 2.554269790649414,
 'test_de_runtime': 5.6729,
 'test_de_samples_per_second': 45.832,
 'test_de_steps_per_second': 5.817,
 'epoch': 8.0}

In [ ]:
lang = 'en(de)'

trainer.evaluate(metric_key_prefix='test_en(de)',
                eval_dataset=en_test_dataset)

***** Running Evaluation *****
  Num examples = 2155
  Batch size = 8
/home/VD/kaveri/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/VD/kaveri/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'test_en(de)_loss': 3.063689947128296,
 'test_en(de)_accuracy@en(de)': 0.19443155452436195,
 'test_en(de)_f1@en(de)': 0.08780287265168876,
 'test_en(de)_precision@en(de)': 0.14753135317786634,
 'test_en(de)_recall@en(de)': 0.0933562511176029,
 'test_en(de)_loss@en(de)': 3.063689947128296,
 'test_en(de)_runtime': 29.4497,
 'test_en(de)_samples_per_second': 73.176,
 'test_en(de)_steps_per_second': 9.168,
 'epoch': 8.0}

wandb: Network error (ReadTimeout), entering retry loop.
wandb: ERROR Error while calling W&B API: internal database error (<Response [500]>)


You should get an overall accuracy of about 56 which is on-par with full finetuning on COPA only but below the state-of-the-art which is sequentially finetuned on an additional dataset before finetuning on COPA.

For results on different languages and a sequential finetuning setup which yields better results, make sure to check out [the MAD-X paper](https://arxiv.org/pdf/2005.00052.pdf).

So you mean to say that even the SOTAs struggle to match randomized baseline for Zh.